##Dating the chaetdontid tree using PAML

from Joey
> Figure 2. A fossil calibrated chronogram of select Chaetodontidae species based on analysis of UCE data. Mike Alfaro add in details on tree program parameters and dating.



>Outgroup
1)     Platax orbicularis
2)     Zanclus cornutus
3)     Add Pygoplites diacanthus
4)     Add another Pomacanthidae if available (Pomacanthus annularis, Pomacanthus rhomboides, Pomacanthus sexstriatus)
5)     Add any other species that would facilitate fossil calibration (Kyphosus vaigiensis, Tilodon sexfasciatus, Scatophagus argus, Selenotoca multifasciata)

>Validation
1)     Root Chaetodon genus to 7 MA (Fessler and Westneat 2007)
2)     Root age of the outgroup node at 65 million years old (K/T boundary), the transition between Mesozoic and Cenozoic marine fish faunas  (Fessler and Westneat 2007)
3)     Hsu et al. 2007 did not use fossil dating to calibrate
4)     Bellwood et al. 2010 placed an exponential prior contraining the Platax node with a hard lower bound age of 50 Ma and a 95% soft upper bound of 65 Ma
5)     Terminal Tethyan Event and closure of Isthmus of Panama too uncertain to use as calibration point

from Matt

179,tholichthys' larva,29.62,n/a,n/a,66,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 29.62"
188,Proacanthurus,49,n/a,n/a,69.3,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 49"
189,Luvarus,54.17,n/a,n/a,80.8,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17"


In [1]:
import sys,os,os.path
sys.path.append(os.path.expanduser('~/Dropbox/tools'))
import glob 
##see script in Dropbox/tools for these functions
from phy_utils import setTreeStyle, rankify, facify, getPamlPars, writeCTL, makeGroups, random_color, rgb2hex, hls2hex,mylayout, calibrations_layout, attachSupport, makeSupportSymbols, addPamlCalibration, makePamlTree, annotatePaml, annotateAndReturn, writePamlTree, copyPamlAlignment, writePamlCtl    
##functions for generating random colors
from ete2 import Tree, faces, TreeStyle, NodeStyle, CircleFace
import colorsys
from numpy import random
from __future__ import division
from collections import OrderedDict
import random
import os, uuid
from ast import literal_eval
from ete2 import Tree, TreeStyle, AttrFace, NodeStyle
import pandas as pd
##dating the bayes tree only
import re
#making the acanthomorph tree look good

def chaetLayout(node):
    #width is set to 2 to increase visibility in large trees when exporting
    node.img_style["hz_line_width"]=4
    node.img_style["vt_line_width"]=4
    if node.is_leaf():
        node.img_style["size"] = 0
        color = "#000000"
        if node.casSpecies:
            faces.add_face_to_node( faces.TextFace(node.casSpecies, ftype="Arial", fsize=20, fstyle = "italic", fgcolor=color), node, 0 )
        if node.name:
            faces.add_face_to_node( faces.TextFace(node.name, ftype="Arial", fsize=10, fstyle = "italic", fgcolor=color), node, 0 )
        if node.family:
            faces.add_face_to_node( faces.TextFace(node.family, ftype="Arial", fsize=12, fgcolor=color), node, 0 )

        if node.up is None:
            node.img_style["size"]=0
            node.dist = 0.35 # you may need to change this value to fit the aspect of your tree
            node.img_style["hz_line_color"] = "#ffffff"
            if UNROOTED:
                node.img_style["vt_line_type"] = 0.001

    else:
        #node.img_style["size"] = 0
        #node.img_style["shape"] = "circle"
        #node.img_style["fgcolor"] = "#000000"
        
        num = node.nodenumber
        faces.add_face_to_node(faces.TextFace(num, ftype="Arial", fsize=12, fgcolor="purple"), node, 0, position = "branch-right")
        faces.add_face_to_node(faces.TextFace(node.support, ftype="Arial", fsize=6, fgcolor="red"), node, 0, position = "branch-right")

        # is the internal node an order?

home = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/"
treepath = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/final-exa-trees/"
resultsdir = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/"
treename = "ExaBayes_ConsensusExtendedMajorityRuleNewick.final.tre"
outfile = "chaet-numbered.tre"

os.chdir(home)


#1 read in tree
tt = Tree(treepath + treename)
outgroup = "Zanclus_cornutus"


tt.set_outgroup(outgroup)
#2 correct names
#read in taxonomy from catalog of fishes
cas = pd.read_csv("/Users/michael_alfaro/Dropbox/fish-taxonomy/AllFish_COF.csv")


pattern = r"([A-Za-z]+\s[A-Za-z]+)(\d*[A-Za-z]*)$"

# changing the taxon names is only ok for the displayed trees.
# it is important to keep the original tip names for the alignment

matched = []
notmatched = []

for tip in tt.get_leaf_names():
    if "_" in tip:
        species =  tip.replace("_", " ").capitalize()
    else: 
        species = tip
    #check for numbers at ends of species name
    #group 1 should be the species without numbers
    #group 2 should be the number
    matchids = re.match(pattern, species)
    if matchids: #this should match everything now
        species = matchids.group(1)
        suffix = matchids.group(2)
    match = cas.loc[cas['species'] == species]
    if match.empty:
        notmatched.append(tip)
    else:
        matched.append(tip)


In [2]:
notmatched


['Chaetodon_plebius',
 'Chaetodon_pelawensis',
 'Chaetodon_punctofasciatus1a',
 'Chaetodon_melanottus1',
 'Chaetodon_melanottus2',
 'Chaetodon_vaganbundus2',
 'Chaetodon_deccusatus']

In [3]:
#print "there are {} unmatched and {} matched species in the tree".format(len(notmatched), len(matched))  
#print notmatched
correctedName = ["Chaetodon plebeius", "Chaetodon pelewensis", "Chaetodon punctatofasciatus1a", "Chaetodon melannotus1", 
                 "Chaetodon melannotus2" ,"Chaetodon vagabundus2", "Chaetodon decussatus"]

fixedNames = dict(zip(notmatched, correctedName))

namesmap = {} #dictionary of originalnames to fixed names

for node in tt.traverse():
    #print "here"
    if node.is_leaf():
        #print node.name
        if "_" in node.name:
            tempname =  node.name.replace("_", " ").capitalize()
            namesmap[node.name] = tempname
            #print tempname
        else:
            tempname = node.name.capitalize()
            namesmap[node.name] = tempname
            #print tempname
        if node.name in fixedNames.keys():
            tempname = fixedNames[node.name]
            namesmap[node.name] = tempname
            #print tempname
        #if I set node.name to temp name, the names are fixed BUT do not correspond to the alignment files
        # this is bad
        # add feature
        #node.name = namesmap[node.name]

        
#3 write tree
#commenting this out for now. If needed, loop through the tree and set each leaf name to be the namesmap value: node.name = namesmap[node.name]


#tt.write(outfile ="chaet-exa-fixed-names.tre")

#tt = Tree("chaet-exa-fixed-names.tre")
#outgroup = "Zanclus cornutus"
#tt.set_outgroup(outgroup)
counter = 0
for node in tt.traverse("postorder"):
    node.add_features( nodenumber = str(counter) )# for calibrations
    counter += 1
    if node.is_leaf():
        node.add_features(family = None)
        node.add_features(genus = None)
        node.add_features(casSpecies = None)

pattern = r"([A-Za-z]+\s[A-Za-z]+)(\d*[A-Za-z]*)$"


for node in tt.traverse():
    if node.is_leaf():
        tax_rec_name = namesmap[node.name] # takes care of misspelled names, numbers could still be present
        matchids = re.match(pattern, tax_rec_name) # seperates numbers from names before trying to match
        if matchids:
            # want a name without underscores or numbers that will match cas
            # tips have underscores
            # tips may have numbers
            # tips may be misspelled
            fname = matchids.group(1) #this name will match the corrected tip, minus any numbers
            cas_match = cas.loc[cas['species'] == fname] #look for the species in CAS
            #print fname
        else:
            print "no match", node.name, 
            #cas_match = cas.loc[cas['species'] == node.name]         
        
        #print cas_match
        ff = cas_match.iloc[0]["family"]
        #print ff
        gg = cas_match.iloc[0]["genus"]
        sp = cas_match.iloc[0]["species"]
        #print "{} {} {} ".format(family, genus, sp)
        node.add_features(family = ff, genus = gg, casSpecies = sp)
        
tt.render(resultsdir +"tempChaets.pdf",  tree_style=setTreeStyle("rect", chaetLayout), w=1200)
tt.render(resultsdir +"11_22_chaets_numbered_nodes.pdf",  tree_style=setTreeStyle("rect", chaetLayout), w=1200)

{'faces': [[661.9266055045871,
   4616.97247706422,
   957.7981651376147,
   4645.871559633028,
   159,
   'Chaetodon semilarvatus'],
  [661.9266055045871,
   4645.871559633028,
   821.5596330275229,
   4659.633027522936,
   159,
   'Chaetodon_semilarvatus1'],
  [661.9266055045871,
   4659.633027522936,
   778.8990825688073,
   4676.146788990825,
   159,
   'Chaetodontidae'],
  [719.7247706422019,
   5514.392201834862,
   747.2477064220184,
   5530.905963302752,
   186,
   '179'],
  [719.7247706422019,
   5530.905963302752,
   730.7339449541284,
   5539.162844036697,
   186,
   '1.0'],
  [538.0733944954128,
   2792.373853211009,
   555.9633027522935,
   2808.887614678899,
   94,
   '90'],
  [538.0733944954128,
   2808.887614678899,
   549.0825688073394,
   2817.1444954128438,
   94,
   '1.0'],
  [608.256880733945,
   4084.4036697247707,
   893.1192660550458,
   4113.302752293578,
   139,
   'Chaetodon melannotus'],
  [608.256880733945,
   4113.302752293578,
   758.256880733945,
   4127

##Based on 11_20_chaets_numbered_nodes.pdf I am calibrating these nodes
![fig](files/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/11_22_chaets_numbered_nodes.pdf)

```
203,Luvarus,54.17,n/a,n/a,80.8,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17"
3,Proacanthurus,49,n/a,n/a,69.3,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 49"
202,tholichthys' larva,29.62,n/a,n/a,66,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 29.62"
```

###taxa to use for species dating
 "Chaetodon_auriga1", "Chaetodon_auripes", "Chaetodon_austriacus2", "Chaetodon_baronessa", "Chaetodon_bennetti2", "Chaetodon_collare1", "Chaetodon_deccusatus", "Chaetodon_dialeucos1", "Chaetodon_falcula", "Chaetodon_fasciatus1", "Chaetodon_gardineri", "Chaetodon_guttatissimus1a", "Chaetodon_kleinii2", "Chaetodon_larvatus1", "Chaetodon_leucopleura1", "Chaetodon_lineolatus1", "Chaetodon_lunula2", "Chaetodon_lunulatus", "Chaetodon_madagaskariensis", "Chaetodon_melanottus1", "Chaetodon_melapterus1", "Chaetodon_mertensii", "Chaetodon_mesoleucos1", "Chaetodon_nigropunctatus", "Chaetodon_oxycephalus1a", "Chaetodon_paucifasciatus2", "Chaetodon_pelawensis", "Chaetodon_pictus1a", "Chaetodon_plebius", "Chaetodon_punctofasciatus1a", "Chaetodon_semilarvatus1", "Chaetodon_speculum1a", "Chaetodon_triangulum1a", "Chaetodon_trichrous", "Chaetodon_trifascialis1", "Chaetodon_trifasciatus", "Chaetodon_unimaculatus1a", "Chaetodon_vagabundus1a", "Chaetodon_xanthurus2b", "Chaetodon_zanzibarensis2", "Forcipiger_flavissimus2", "Forcipiger_longirostris1a", "Heniochus_acuminatus2b", "Heniochus_diphreutes2b", "Heniochus_intermedius", "Platax_orbicularis1", "Prognathodes_marcellae", "Prognathodes_aculeatus", "Zanclus_cornutus", "naso_unicornis", "pomacanthus_paru", "acanthurus_olivaceus"
 
 



Now take the pruned tree and read in the calibrations file.

check to make sure the node numbers on the pruned tree still make sense

**not doing the pruned analysis for now although number do still correpsond to full tree**

In [4]:
#pruned_tree.render(resultsdir +"11_22_pruned_chaets_numbered_nodes.pdf",  tree_style=setTreeStyle("rect", chaetLayout), w=1200)

#read in a calibration with root fixed at 66 my
calibrations_path = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/1_calibration_bounds copy.csv"
basmldir = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/baseml-chaets/"
#this tree still looks good. Now 

scheme1, scheme2, scheme3 = {}, {}, {}

dd = pd.read_csv(calibrations_path, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)
#populate the scheme dictionaries
#Matt gave alternative calibratiosn for these nodes from different sources. For the paper I am using lower and upper bounds.
for cal_node in dd.index:
    min_age = dd.loc[cal_node][1] 
    max_age_1 = dd.loc[cal_node][2]
    max_age_2 = dd.loc[cal_node][3]
    max_age_3 = dd.loc[cal_node][4]
    
    scheme1[cal_node] = [min_age, max_age_1]
    scheme2[cal_node] = [min_age, max_age_2]
    scheme3[cal_node] = [min_age, max_age_3]

schemes = [scheme3]
#tree_title = "1_cali_fixed.tre"
##Run on the full tree just to make sure the numbering scheme makes sense
tree_title = "1_cali_fixed_full_tre"
numtaxa = len(tt.get_leaf_names())
numtrees = 1 # change this if ever considering multiple topologies
calibrations_path = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/1_calibration_bounds copy.csv"
basmldir = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/baseml-chaets/"
pst = annotatePaml(tt, scheme3, raw = False)

full_baseml_paml_tree = writePamlTree(pst, tree_title, numtaxa, numtrees, return_tree = True )
#write the calibration map for the pruned tree
tt.render(basmldir +"full{}.pdf".format(tree_title), tree_style=setTreeStyle("rect", calibrations_layout), w=1200)   


this is the treestring before temptree
temptree (Zanclus_cornutus,((acanthurus_olivaceus,naso_unicornis),((pomacanthus_paru,(Platax_orbicularis1,Platax_orbicularis2)),(((Heniochus_diphreutes1a,((Heniochus_intermedius,Heniochus_diphreutes2b),(Heniochus_diphreutes,Heniochus_acuminatus2b))),((Forcipiger_longirostris1a,Forcipiger_longirostris),(Forcipiger_flavissimus1a,(Forcipiger_flavissimus1,(Forcipiger_flavissimus2,(Forcipiger_flavissimus3c,Forcipiger_flavissimus2b)))))),((Prognathodes_aculeatus,Prognathodes_marcellae),(((Chaetodon_trifasciatus,(Chaetodon_lunulatus,(Chaetodon_austriacus2,(Chaetodon_austriacus1,(Chaetodon_melapterus2,Chaetodon_melapterus1))))),(((Chaetodon_larvatus2,Chaetodon_larvatus1),(Chaetodon_baronessa,(Chaetodon_triangulum2b,Chaetodon_triangulum1a))),((Chaetodon_trifascialis1a,(Chaetodon_trifascialis2,Chaetodon_trifascialis1)),(Chaetodon_plebius,((Chaetodon_bennetti1,Chaetodon_bennetti2),((Chaetodon_zanzibarensis1,Chaetodon_zanzibarensis2),(Chaetodon_speculum2b,Cha

{'faces': [[726.7452402538531,
   1783.1368993653673,
   979.1477787851315,
   1805.9836808703537,
   159,
   'Chaetodon_semilarvatus1'],
  [773.526745240254,
   2125.158658204896,
   802.9011786038078,
   2143.6536718041707,
   186,
   '179'],
  [611.4233907524932,
   1074.2067089755215,
   631.0063463281958,
   1092.7017225747961,
   94,
   '90'],
  [679.9637352674524,
   1577.5158658204896,
   917.1350861287399,
   1600.362647325476,
   139,
   'Chaetodon_melanottus2'],
  [867.0897552130554,
   2217.225747960109,
   1054.2157751586583,
   2240.072529465095,
   196,
   'Chaetodon_pictus1'],
  [537.4433363553944,
   1002.096554850408,
   557.026291931097,
   1020.5915684496828,
   84,
   '92'],
  [463.46328195829557,
   174.61468721668177,
   483.04623753399824,
   193.1097008159565,
   17,
   '12'],
  [574.4333635539438,
   957.1169537624661,
   594.0163191296465,
   975.6119673617408,
   85,
   '80'],
  [611.4233907524932,
   974.2520398912059,
   631.0063463281958,
   992.747053490

#dating on the pruned tree
To check and see what happens if we include on the "species" level divergence we need a tree and an alignment that matches the tree



In [5]:
# import os
# #now joey's list should have the old taxonomy
# species_only = ["Chaetodon_auriga1", "Chaetodon_auripes", "Chaetodon_austriacus2", "Chaetodon_baronessa", "Chaetodon_bennetti2", "Chaetodon_collare1", 
#                 "Chaetodon_deccusatus", "Chaetodon_dialeucos1", "Chaetodon_falcula", "Chaetodon_fasciatus1", "Chaetodon_gardineri", "Chaetodon_guttatissimus1a", 
#                 "Chaetodon_kleinii2", "Chaetodon_larvatus1", "Chaetodon_leucopleura1", "Chaetodon_lineolatus1", "Chaetodon_lunula2", "Chaetodon_lunulatus", "Chaetodon_madagaskariensis", 
#                 "Chaetodon_melanottus1", "Chaetodon_melapterus1", "Chaetodon_mertensii", "Chaetodon_mesoleucos1", "Chaetodon_nigropunctatus", "Chaetodon_oxycephalus1a", "Chaetodon_paucifasciatus2", 
#                 "Chaetodon_pelawensis", "Chaetodon_pictus1a", "Chaetodon_plebius", "Chaetodon_punctofasciatus1a", "Chaetodon_semilarvatus1", "Chaetodon_speculum1a", 
#                 "Chaetodon_triangulum1a", "Chaetodon_trichrous", "Chaetodon_trifascialis1", "Chaetodon_trifasciatus", "Chaetodon_unimaculatus1a", "Chaetodon_vagabundus1a", 
#                 "Chaetodon_xanthurus2b", "Chaetodon_zanzibarensis2", "Forcipiger_flavissimus2", "Forcipiger_longirostris1a", "Heniochus_acuminatus2b", "Heniochus_diphreutes2b", 
#                 "Heniochus_intermedius", "Platax_orbicularis1", "Prognathodes_marcellae", "Prognathodes_aculeatus", "Zanclus_cornutus", "naso_unicornis", "pomacanthus_paru", "acanthurus_olivaceus"]

        
# #first delete taxa from tree so that SPECIES are represented
# # remeber in ETE2 pruned takes a list of species to RETAIN not drop!
# tips = tt.get_leaf_names()
# dropset = set(tips) - set(species_only)

# print "there are {} tips in the tree, {} species reps and {} taxa to drop".format(len(tips),  len(species_only), len(dropset))
# set(species_only) - set(tips)                                                                              
# pruned_tree = tt.copy()
# pruned_tree.prune(species_only)
# pruned_tree.describe()


# alignpath = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/alignments/mafft-internal-trimmed-clean-75p.phylip"

# seqdict = {}
# with open(alignpath) as ff:
#     aligndim = ff.readline()
#     for line in ff:
#         taxon, seq = line.split("  ")
#         seqdict[taxon] = seq

# nspecies, nchars = aligndim.split()
# prunedtips = pruned_tree.get_leaf_names()
# nspecies_pruned = len(prunedtips)
# pruned_align_dim = "{} {}\n".format(nspecies_pruned, nchars)



# fullout = open("pruned-alignment-chaets.phy", "w")
# fullout.write(pruned_align_dim) #first line of alignment file with pruned species number
# for taxon in sorted(prunedtips):
#     seq = seqdict[taxon] # get the sequence foe each pruned tip
#     outline = "{}  {}".format(taxon, seq) #changing name here
#     fullout.write(outline)
# fullout.close

# #need to write the pruned tree file with the calibrations


##Baseml results
I ran HKY and Gamma models using baseml. 

HKY mean rate per 10 million years = 0.005390
GTR mean rate per 10 million years = 0.005396

Following PAML documentation, we set $\alpha$ = 2 and $\alpha$ / $\beta to be equal to the mean rate.

alpha/beta = r

beta = alpha/r

beta = 2/0.005390

beta = 
```{r}
r0 = 0.005396
t0 = 10
tnew = 10
k = tnew / t0
rnew = r0/k
rnew
alpha = 2
beta_old = 20.372
beta_new = k * beta_old
beta_new
```


##11-26-2015 dating chaets
OK I am going to try the 3 fossils from the acanthomorph paper plus *C. ficheuri* from Carnevale 2006. THis fossil probably dates a node within chaets but defintely constrains crown chaets to be at least 7 MY years old. I am setting the upper bound to be 65 MY 

Node,Taxon,Minimum,Published 95% CI (Benton et al. 2015),Published 95% CI (Friedman et al. 2014),New: Estimated 95% CI,"Near et al. (2012, 2013)",Benton et al. (2015),Hedman age sequence
203,Luvarus,54.17,n/a,n/a,80.8,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17"
3,Proacanthurus,49,n/a,n/a,69.3,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 49"
202,tholichthys' larva,29.62,n/a,n/a,66,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 29.62"
199,ficheuri,7.0,n/a,n/a,66,Yes*,No,"98, 93.9, 69.71, 55.2, 54.17, 29.62"


In [6]:
def writeCTL(template_ctl, ctl_name = "input-ctl"):
    #writes PAML control file
    # if hessian == "hessian":
    #     template_ctl["usedata"] = "3"
    # elif hessian == "post-hessian":
    #     template_ctl["usedata"] = "2"
    # else:
    #     print "\nneed information for usedata argument\n"
    #number_partitions = partitions
    #template_ctl["seqfile"] = os.path.basename(partitions)
    #template_ctl["treefile"] = tree_title
    #n1 = number_partitions + "_partitions"
    #n2 = tree_title
    #n3 = "_run_{}".format(run)
    #template_ctl["outfile"] = "{}_{}_{}.out".format(n1, n2, n3)
    #template_ctl["ndata"] = number_partitions
    #print "\nNew file"
    print len(template_ctl.keys())
    #ctl_name = "caringimorph.ctl" #"ctl_{}_{}_{}_{}.ctl".format(hessian, n3, n1, n2 )

    #helper function to write ctl files, takes a dictionary of parameters to update (pars)
    temp_ctl = r"""
              seed = {seed}
           seqfile = {seqfile}
          treefile = {treefile}
           outfile = {outfile}

             ndata = {ndata}
           seqtype = {seqtype}    * 0: nucleotides; 1:codons; 2:AAs
           usedata = {usedata}    * 0: no data; 1:seq like; 2:normal approximation; 3:out.BV (in.BV)
             clock = {clock}    * 1: global clock; 2: independent rates; 3: correlated rates
           RootAge = {RootAge}  * safe constraint on root age, used if no fossil for root.

             model = {model}    * 0:JC69, 1:K80, 2:F81, 3:F84, 4:HKY85
             alpha = {alpha}    * alpha for gamma rates at sites
             ncatG = {ncatG}   * No. categories in discrete gamma

         cleandata = {cleandata}    * remove sites with ambiguity data (1:yes, 0:no)?

           BDparas = {BDparas}   * birth, death, sampling
       kappa_gamma = {kappa_gamma}      * gamma prior for kappa
       alpha_gamma = {alpha_gamma}      * gamma prior for alpha

       rgene_gamma = {rgene_gamma}    * gammaDir prior for rate for genes
      sigma2_gamma = {sigma2_gamma}  * gammaDir prior for sigma^2     (for clock=2 or 3)

          finetune = {finetune} * auto (0 or 1): times, rates, mixing, paras, RateParas, FossilErr

             print = {print}
            burnin = {burnin}
          sampfreq = {sampfreq}
           nsample = {nsample}

    *** Note: Make your window wider (100 columns) before running the program.
    """
    
    populated_ctl = temp_ctl.format(**template_ctl)
    
    with open(ctl_name, "w") as ff:
            ff.write(populated_ctl) 
            ff.close()

In [14]:
%load_ext autoreload
%autoreload 2
#write the paml config file
import os
import glob
import shutil
from phy_utils import getPamlPars, writeCTL, calibrations_layout,  addPamlCalibration, makePamlTree, annotatePaml, annotateAndReturn, writePamlTree, copyPamlAlignment, writePamlCtl    

# read in calibrations file
calibrations_path = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/4_calibration_bounds.csv"
paml_path = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/"
#this tree still looks good. Now 

scheme1, scheme2, scheme3 = {}, {}, {}

dd = pd.read_csv(calibrations_path, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)
#populate the scheme dictionaries
#Matt gave alternative calibratiosn for these nodes from different sources. For the paper I am using lower and upper bounds.
for cal_node in dd.index:
    min_age = dd.loc[cal_node][1] 
    max_age_1 = dd.loc[cal_node][2]
    max_age_2 = dd.loc[cal_node][3]
    max_age_3 = dd.loc[cal_node][4]
    
    scheme1[cal_node] = [min_age, max_age_1]
    scheme2[cal_node] = [min_age, max_age_2]
    scheme3[cal_node] = [min_age, max_age_3]

schemes = [scheme3]
#tree_title = "1_cali_fixed.tre"
##Run on the full tree just to make sure the numbering scheme makes sense
tree_title = "4_cali_fixed_full_tre"
numtaxa = len(tt.get_leaf_names())
numtrees = 1 # change this if ever considering multiple topologies

alignpath = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/alignments/" #directory with alighments
outpath = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/" #where run files will be created
workingdir = paml_path #working in this folder

chaets_4_calib = annotatePaml(tt, scheme3, raw = False)

full_baseml_paml_tree = writePamlTree(chaets_4_calib, tree_title, numtaxa, numtrees, return_tree = True )
#write the calibration map for the pruned tree
tt.render(paml_path +"labelled_nodes_{}.pdf".format(tree_title), tree_style=setTreeStyle("rect", calibrations_layout), w=1200) 


###write out PAML for for analysis with Brant's cluster



heshlist = ["hessian", "post-hessian"]
nruns = 10 #number of runs for each analysis using the approximation

##for making the control file (parsi_mcmc.ctl is parameter values I updated after experimenting and emailing Ziheng in June)
ctlfile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/control_file_template/parsi_mcmc.ctl"
###should move this elsewhere--only need to do it once to get the paml ctl structure
with open(ctlfile, 'r') as ff:
    ctl = ff.read()
    ff.close()

###overwrites files eachtime this is called
if not os.path.exists(outpath):
    os.makedirs(outpath)
else:
    shutil.rmtree(outpath)           #removes all the subdirectories!
    os.makedirs(outpath)
    



#aligns = glob.glob(alignpath + "3_part*.*")
aligns = glob.glob(alignpath + "*.phylip")

basectl = getPamlPars(ctl) #suck in base parameters for ctl file

#alignments with parsimony clusters end in sate-gblocks-clean-min-114-taxa-missing-no-outgroups-CLUSTERED-raxml.PAML.phylip.parsi.seq
basectl['ndata'] = "1" #set number of partitions
#basectl['RootAge'] = "'B(9.8, 20.0, 1e-300, 1e-300)'" #my temp upper bound for a safe prior
basectl['print'] = '1000'
basectl['alpha'] = 0.1
basectl['ncatG'] = 5
basectl['BDparas'] = '1 1 0'
basectl['rgene_gamma'] = '2 371.0575 1'

#schemes = [scheme1, scheme2, scheme3]

for alignment in aligns:
    alignFileName = os.path.basename(alignment)
    basectl['seqfile'] = alignFileName #set alignment name in ctl file
    #aligndir = outpath + os.path.basename(alignment).split("_")[0] + "_part/"
    for num, scheme in enumerate(schemes):
        #%qtconsole
        #tree_title = "parsi_scheme_{}.tre".format(num)
        #tree_title = "16_cali_no_outgroups_scheme_{}.tre".format(1) #only looking at scheme 3
        basectl["treefile"] = tree_title
        for hes in heshlist:
            if hes == "post-hessian":
                basectl["usedata"] = "2"
                basectl['burnin'] = '2500'
                basectl['samplefreq'] = '100'
                basectl['nsample'] = '10000'
                for run in range(nruns):
                    basectl["outfile"] = "run-{}-partitions-{}-align-{}.out".format(run + 1, 1, alignFileName)
                    #create a directory for each replicate of the run
                    rep_dir_text = "scheme-{}-HE-{}-run{}".format(num, hes, run+1)
                    rep_path = outpath + "/" + rep_dir_text
                    os.makedirs(rep_path)
                    os.chdir(rep_path)
                    paml_tree_string = annotatePaml(tt, scheme3, raw = False)
                    writePamlTree(paml_tree_string, tree_title, numtaxa, numtrees )
                    copyPamlAlignment(alignment, rep_path)
                    #writePamlCtl(hes, tree_title, alignment, basectl)
                    #change this so that the ctl file is configured before passing in to writeCT:
                    writeCTL( basectl, "chaets.ctl" )
                    
                    """
                    #schemenum = str(num)
                    curdir = aligndir + "scheme_{}_{}_run_{}".format(num, hes, run+1)
                    os.makedirs(curdir)
                    os.chdir(curdir)
                    paml_tree_string = annotatePaml(pruned_tree, scheme)
                    
                    writePamlTree(paml_tree_string, tree_title )
                    copyPamlAlignment(alignment, curdir)
                    #writePamlCtl(hes, tree_title, alignment, basectl)
                    writeCTL(hes, tree_title, alignment, basectl, run+1, )
                    """
            else:
                basectl["outfile"] = "hessian-partitions-{}-align-{}.out".format(1, alignFileName)
                basectl["usedata"] = "3"
                basectl['burnin'] = '3000'
                basectl['samplefreq'] = '100'
                basectl['nsample'] = '7500'
                #schemenum = str(num)
                rep_dir_text = "scheme-{}-HE-{}".format(num, hes)
                rep_path = outpath + "/" + rep_dir_text
                os.makedirs(rep_path)
                os.chdir(rep_path)
                paml_tree_string = annotatePaml(tt, scheme3, raw = False)
                
                writePamlTree(paml_tree_string, tree_title, numtaxa, numtrees )
                copyPamlAlignment(alignment, rep_path)
                    #writePamlCtl(hes, tree_title, alignment, basectl)
                writeCTL(basectl )
    
##OK the alignment names have caps but the tree names do not. Need to fix this.....

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
calibration text is 'B(4.9,6.93,1e-300,0.05)'
calibration text is 'B(2.962,6.6,1e-300,0.05)'
calibration text is 'B(5.417,8.08,1e-300,0.05)'
calibration text is 'B(0.7,6.6,1e-300,0.05)'
***this is the nhx tree***

(Zanclus_cornutus[&&NHX:nodenumber=0],((acanthurus_olivaceus[&&NHX:nodenumber=1],naso_unicornis[&&NHX:nodenumber=2])[&&NHX:nodenumber=3:paml_cal='B_4.9_6.93_1e-300_0.05_'],((pomacanthus_paru[&&NHX:nodenumber=4],(Platax_orbicularis1[&&NHX:nodenumber=5],Platax_orbicularis2[&&NHX:nodenumber=6])[&&NHX:nodenumber=7])[&&NHX:nodenumber=8],(((Heniochus_diphreutes1a[&&NHX:nodenumber=9],((Heniochus_intermedius[&&NHX:nodenumber=10],Heniochus_diphreutes2b[&&NHX:nodenumber=11])[&&NHX:nodenumber=12],(Heniochus_diphreutes[&&NHX:nodenumber=13],Heniochus_acuminatus2b[&&NHX:nodenumber=14])[&&NHX:nodenumber=15])[&&NHX:nodenumber=16])[&&NHX:nodenumber=17],((Forcipiger_longirostris1a[&&NHX:nodenumber=18],Forci

In [15]:
##copy the hessian to the post hessian folders
import shutil, glob
#generates a line of code to execute on Brant's clusters

#workdir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/caringimorph_dating/pamlfiles/"
workdir = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/"
rundirs = [os.path.join(workdir,o) for o in os.listdir(workdir) if os.path.isdir(os.path.join(workdir,o))]
os.chdir(workdir)

print rundirs
bv_path = workdir + "scheme-0-HE-hessian/out.BV"
for rdir in rundirs:
    #pamldirs = [os.path.join(rdir,o) for o in os.listdir(rdir) if os.path.isdir(os.path.join(rdir,o))]
    #print pamldirs
    #bv_path = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/pars-10-cali-out.BV"
    #bv_path = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_10_calibrations/PAML_run_files/3_partitions/scheme-0-HE-hessian/out.BV"
    

    shutil.copy2(bv_path, rdir + "/in.BV")
    #os.remove(pp + "/out.BV")
    #print pp
    runline = "mcmctree {}".format(os.path.basename(glob.glob(rdir + "/*.ctl")[0]))
    os.chdir(rdir)
    with open("run_command.txt", "w") as ff:
        ff.write(runline)
    ff.close()
    #%qtconsole
    


['/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-hessian', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-post-hessian-run1', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-post-hessian-run10', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-post-hessian-run2', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-post-hessian-run3', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-post-hessian-run4', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-run-files/scheme-0-HE-post-hessian-run5', '/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/cha

copy the 10 post hessian directories to yog and run with paralles

```
parallel --results . "cd {//} && bash {/}" ::: scheme-0-HE-post-hessian-run*/run_command.txt
```
 tried to do this but JC needs to add me to the sudoers list. Once that happens do this:
 
 ```
sudo apt-get install parallel
sudo rm /etc/parallel/config
```

In [16]:

import sys,os,os.path
sys.path.append(os.path.expanduser('~/Dropbox/tools'))

import glob
import shutil
import os
import dendropy
import StringIO
import pandas as pd

from phy_utils import updatePamlAges

%load_ext autoreload
%autoreload 2

#parentdir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/PAML_run_files/3_partitions/12_cali_3_parts_paml-output/"
#resultsdir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/PAML_run_files/3_partitions/12_cali_3_parts_paml-output/combined-output/"

##changing to process carangimorphs
parentdir = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/"
resultsdir = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/combined-output/"


#logname = "mcmc.log"
logname = "mcmc.txt"
counter = 0
logfiles = []
for r,d,f in os.walk(parentdir):
    if logname in f:
        print r, "\n",
        outname = "run-{}-".format(counter) + logname
        print logname, outname , "\n"
        shutil.copy2(os.path.join(r, logname), resultsdir + outname)
        counter += 1
        logfiles.append(outname) #list of all outfiles, will pass to log combiner
        #break
    #print r
    #print glob.glob("*.log")
#%qtconsole



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/scheme-0-HE-post-hessian-run1 
mcmc.txt run-0-mcmc.txt 

/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/scheme-0-HE-post-hessian-run10 
mcmc.txt run-1-mcmc.txt 

/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/scheme-0-HE-post-hessian-run2 
mcmc.txt run-2-mcmc.txt 

/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/scheme-0-HE-post-hessian-run3 
mcmc.txt run-3-mcmc.txt 

/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/scheme-0-HE-post-hessian-run4 
mcmc.txt run-4-mcmc.txt 

/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/scheme-0-HE-post-hessian-run5 
mcmc.tx

In [17]:
##need to run these longer and the ages look too old to me. Just running this to send to Joey

#%qtconsole
#remove logfile1 and 2--Tracer inspection shows a lack of convergence for n167-170
#logfiles.remove('run-1-mcmc.log')
#logfiles.remove('run-2-mcmc.log')
print logfiles

##ome convergence problems. Looks like runs 0,1,6,7 converged

badruns = [2,3,4,5,8,9]
for run in badruns:
    bad = ("run-{}-mcmc.txt".format(run))
    if bad in logfiles:
        logfiles.remove(bad)
    
os.chdir(resultsdir)

pdir = {}

pdir["beastpath"] = "/Applications/beAST/lib/beast.jar"
pdir["outfilename"] = "chaets-4-cal-4-part-thinned-10-runs-combined.log"
pdir["resample"] = 4
pdir["burnin"] = 1250000 #following For the LOG FILE - the number you put into the burnin box will be ---- 
#> C*B. Or in my case 50,000,000*0.1 = 5,000,000 

"""For the TREE FILE - number you put into the burnin box will be -----> 
(C/L)*B. Or in my case (50,000,000/10,000)*0.1 = 500 """
pdir["logfiles"] =  ' '.join(str(ll) for ll in logfiles) 



runLC = "java -classpath {beastpath} dr.app.tools.LogCombiner -burnin {burnin} -resample {resample} -renumber {logfiles} {outfilename}".format(**pdir)

os.system(runLC)

['run-0-mcmc.txt', 'run-1-mcmc.txt', 'run-2-mcmc.txt', 'run-3-mcmc.txt', 'run-4-mcmc.txt', 'run-5-mcmc.txt', 'run-6-mcmc.txt', 'run-7-mcmc.txt', 'run-8-mcmc.txt', 'run-9-mcmc.txt']


0

Use tracer to make a new csv from combined.out and update the label tree


In [18]:
#update ages
#get paths to tracer data file, paml label tree as a string


#tree_string = open("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/PAML_run_files/3_partitions/12_cali_3_parts_paml-output/3-part-paml-labels.tree").read()
#tracer_path = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/PAML_run_files/3_partitions/12_cali_3_parts_paml-output/combined-8-run-3-part.csv"
#outfile = "3-part-12-cali-8-run-consensus-ages.tre"

labeltree = open("/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/chaet-label-tree.tre").read()
tracer_results_path = "/Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/chaet-combined-tracer.csv"
outfile = "chaet-1-part-4-cali-10-run-consensus-ages.tre"
os.chdir(resultsdir)

# %qtconsole
updatePamlAges(tracer_results_path, labeltree, outfile)


104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
writing tree to file /Users/michael_alfaro/Dropbox/chaedontidae_analysis/chaet-dating-results/chaets-paml/paml-results/combined-output/chaet-1-part-4-cali-10-run-consensus-ages.tre
